In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
tflite_model_name = 'rnn_lite_model.tflite'
model_name = 'rnn_pos_model'

converter = tf.lite.TFLiteConverter.from_saved_model(model_name) 
tflite_model = converter.convert()

In [3]:
import joblib
joblib.dump(tflite_model,tflite_model_name)

['rnn_lite_model.tflite']

#### Loading and Testing the tflite model

In [4]:
# import os
# os.chdir(save_path)
# print(os.getcwd())

import joblib
tags_dict = joblib.load('tags.joblib')
word_tokenizer = joblib.load('tokenizer.joblib')
tflite_model = joblib.load('rnn_lite_model.tflite')


print('\n')
print(tags_dict)



{1: 'noun', 2: 'verb', 3: '.', 4: 'adp', 5: 'det', 6: 'adj', 7: 'adv', 8: 'pron', 9: 'conj', 10: 'prt', 11: 'num', 12: 'x', 0: 'padding'}


In [5]:
interpreter = tf.lite.Interpreter(model_content = tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details= interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 125]
Input Type: <class 'numpy.float32'>
Output Shape: [  1 125  13]
Output Type: <class 'numpy.float32'>


In [6]:
import re
import numpy as np

def text_preprocess(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.rstrip()
    text = text.lstrip()
    return text

def predict(text, max_len, tags_dict, word_tokenizer):

    original_text = text
    text = text_preprocess(text)
    text = text.split(' ')
    words = word_tokenizer.texts_to_sequences(text)
    words_dict = word_tokenizer.word_index
    words_dict = dict([(value, key) for key, value in words_dict.items()])
    words = np.array(words)
    words = words.ravel()

    words = tf.keras.preprocessing.sequence.pad_sequences(
        [words], maxlen = max_len, padding = 'pre',
        truncating='post', value=0.0 )
  
    words = np.array(words)
    original_words = words
    words = words.astype('float32')
    
    interpreter.set_tensor(input_details[0]['index'], words)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details[0]['index'])
    pred_prob = np.max(pred,axis=2)
    pred = np.argmax(pred,axis=2)

    pred = pred.ravel()
    pred = np.trim_zeros(pred, 'f')
    pred_prob = pred_prob.ravel()
    pred_prob = pred_prob[(pred_prob.shape[0] - pred.shape[0]):]
    original_words = original_words.ravel()
    original_words = np.trim_zeros(original_words, 'f')

    output = []
    for i in range(len(pred)):
        temp = []
        if words_dict[original_words[i]] == '<OOV>':
            temp.append(text[i])
        else:
            temp.append(words_dict[original_words[i]])
        temp.append(tags_dict[pred[i]])
        temp.append(pred_prob[i])
        output.append(temp)

    return original_text, output

In [7]:
max_len = 125
text = 'my favourite movie is spiderman'

my_text, output = predict(text, max_len,tags_dict, word_tokenizer)

(1, 125)


In [8]:
my_text

'my favourite movie is spiderman'

In [9]:
output

[['my', 'det', 0.81986994],
 ['favourite', 'noun', 0.8906838],
 ['movie', 'noun', 0.9999763],
 ['is', 'verb', 0.9998092],
 ['spiderman', 'noun', 0.6483571]]

In [10]:
from pywebio.input import input,TEXT
from pywebio.output import put_tabs,put_table

In [11]:
def predict_output():
    text = input('Parts Of Speech Tagger',placeholder = "Enter the text", type = TEXT)
    
    max_len = 125
    text, output = predict(text, max_len,tags_dict, word_tokenizer)
    
    put_tabs([
    {'title': 'Results', 'content': [
        put_table(output, header=['Words', 'Tag', 'Probability']) ]},
    {'title': 'Input text', 'content': text}
    ])

In [12]:
predict_output()

(1, 125)
